In [22]:
!pip install -q langgraph langchain langchain-community langchain-core langchain-openai chromadb pypdf sentence-transformers langchain-ollama
print("Packages installed successfully!")

Packages installed successfully!


In [23]:
    !sudo apt-get install -y pciutils
    !curl -fsSL https://ollama.com/install.sh | sh


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [24]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Run Ollama server in background
import subprocess
subprocess.Popen(["ollama", "serve"])

# Pull the embedding model (run once)
!ollama pull all-minilm

print("Ollama installed and running! Model pulled.")
# Wait for server to start (optional sleep)
import time
time.sleep(10)  # Give time for startup

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

Ollama installed and running! Model pulled.


In [25]:
    import subprocess
    import os

    def start_ollama_server():
        os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
        os.environ['OLLAMA_ORIGINS'] = '*'
        subprocess.Popen(["ollama", "serve"])
        print("🚀 Ollama server launched successfully!")

    start_ollama_server()

🚀 Ollama server launched successfully!


In [26]:
# ========= Cell 2: Imports & Config ==========
import os
import uuid
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END
from langchain.schema import Document

# Global in-memory cache for session vectorstores
SESSION_PDF_STORES: dict = {}

# --- Keep your token lines unchanged as requested ---
os.environ["GITHUB_TOKEN"] = "your_github_api_key"
token = os.environ.get("GITHUB_TOKEN")
if not token:
    raise ValueError("GITHUB_TOKEN not set.")

# Configure LLM (use your provided base_url/model)
llm = ChatOpenAI(
    api_key=token,
    base_url="https://models.github.ai/inference",
    model="openai/gpt-4.1-nano",
    temperature=0.0
)

# Configure embeddings (Ollama)
embeddings = OllamaEmbeddings(model="all-minilm")

print("LLM and Embeddings configured successfully!")

# Sanity tests (non-fatal)
try:
    test_response = llm.invoke("Hello, what is 2+2?").content
    print("LLM Test Response:", test_response)
except Exception as e:
    print("LLM test failed (ok if service not reachable):", e)

try:
    embedding_test = embeddings.embed_query("Test embedding")
    print("Embeddings Test: Success! Vector length:", len(embedding_test))
except Exception as e:
    print("Embeddings Error (ok if Ollama not running locally):", str(e))

LLM and Embeddings configured successfully!
LLM Test Response: Hello! 2 + 2 equals 4.
Embeddings Test: Success! Vector length: 384


In [27]:
# ========= Cell 3: State & Long-term memory setup ==========
class ResearchState(TypedDict):
    messages: Annotated[List[BaseMessage], "Conversation history (short-term memory)"]
    pdf_paths: Annotated[List[str], "Paths to uploaded PDFs (session-local)"]
    pdf_chunks: Annotated[List[str], "Chunked text from the uploaded PDFs (session-local)"]
    retrieved_docs: Annotated[List[str], "Retrieved relevant documents from PDF or long-term memory"]
    long_term_insights: Annotated[List[str], "New insights to store in long-term memory after response"]
    conversation_count: Annotated[int, "Number of exchanges in current session"]
    session_id: Annotated[str, "Unique session identifier"]  # Explicitly define in state

# Persistent long-term memory directory
long_term_db_path = "/content/long_term_memory"
os.makedirs(long_term_db_path, exist_ok=True)

# Initialize persistent Chroma for long-term memory (embedding_function kw kept)
long_term_vectorstore = Chroma(
    collection_name="long_term_insights",
    embedding_function=embeddings,
    persist_directory=long_term_db_path
)

def persist_long_term():
    """Persist long-term store if supported; tolerate deprecation/auto-persist differences."""
    try:
        if hasattr(long_term_vectorstore, "persist"):
            long_term_vectorstore.persist()
            print("Long-term memory persisted to:", long_term_db_path)
        else:
            print("Long-term memory auto-persist or persist not available in this Chroma version.")
    except Exception as e:
        print("Note: persistence call issued but raised:", e)

print("Long-term memory initialized at:", long_term_db_path)

Long-term memory initialized at: /content/long_term_memory


In [28]:
# ========= Cell 4: PDF Processing & Summarization Helpers ==========
def process_pdf_paths(pdf_paths: List[str], chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Load multiple PDFs and chunk them. Return chunk texts only.
    (Session vectorstores are created separately and cached globally.)
    """
    all_chunks: List[str] = []
    for pdf_path in pdf_paths:
        try:
            loader = PyPDFLoader(pdf_path)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            chunks = text_splitter.split_documents(documents)
            chunk_texts = [d.page_content for d in chunks]
            all_chunks.extend(chunk_texts)
            print(f"Processed {pdf_path}: {len(chunk_texts)} chunks")
        except Exception as e:
            print(f"Error processing {pdf_path}: {e}")
    return all_chunks


def summarize_text_with_llm(text: str, max_sentences: int = 5) -> str:
    """
    Use the LLM to produce a concise summary.
    """
    prompt_template = (
        "Summarize the following text into up to {n} short, numbered sentences. Keep each sentence concise.\n\n"
        "Text:\n{txt}\n\nSummary:"
    )
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()
    try:
        summary = chain.invoke({"n": max_sentences, "txt": text})
        return summary
    except Exception as e:
        return "Summary failed: " + str(e)

In [29]:
# ========= Cell 5: LangGraph Node Implementations ==========
def load_pdfs_node(state: ResearchState) -> ResearchState:
    """
    Load and chunk all PDFs for this session; create a cached session Chroma store and
    ensure session_id is preserved from initial_state.
    """
    pdf_paths = state.get("pdf_paths", []) or []
    # Ensure session_id is present; raise error if not provided in initial_state
    ### MODIFIED: Removed fallback session_id creation to enforce initial_state provision
    session_id = state.get("session_id")
    if not session_id:
        raise ValueError("session_id must be provided in initial_state")

    if not pdf_paths:
        print("No PDFs provided in this session.")
        SESSION_PDF_STORES.setdefault(session_id, None)
        return {**state, "pdf_chunks": [], "pdf_paths": pdf_paths, "session_pdf_store_present": False}

    # Produce chunk texts
    chunks = process_pdf_paths(pdf_paths)
    state = {**state, "pdf_chunks": chunks, "pdf_paths": pdf_paths}

    # Create session vectorstore if not already cached
    if SESSION_PDF_STORES.get(session_id) is None and chunks:
        try:
            session_store = Chroma(collection_name=f"session_pdf_{session_id}", embedding_function=embeddings)
            session_store.add_texts(chunks)
            SESSION_PDF_STORES[session_id] = session_store
            print("Session PDF vectorstore created and cached (via add_texts).")
            state["session_pdf_store_present"] = True
        except Exception as e:
            print("Warning: failed to create session PDF vectorstore (embedding service might be offline):", e)
            SESSION_PDF_STORES[session_id] = None
            state["session_pdf_store_present"] = False
    else:
        state["session_pdf_store_present"] = bool(SESSION_PDF_STORES.get(session_id))
        if state["session_pdf_store_present"]:
            print("Using pre-existing session vectorstore from cache.")

    print(f"load_pdfs_node: loaded {len(chunks)} chunks across {len(pdf_paths)} PDF(s). session_id={session_id}")
    return state


def analyze_query_node(state: ResearchState) -> ResearchState:
    """Analyze query (extract keywords, intent) using LLM."""
    last_message = state["messages"][-1].content if state["messages"] else ""
    prompt = ChatPromptTemplate.from_template(
        "Analyze this user query and return comma-separated keywords and a one-line intent summary.\n\n"
        "Query: {query}\n\nOutput format: keywords: kw1, kw2; intent: one-line summary"
    )
    chain = prompt | llm | StrOutputParser()
    try:
        analysis = chain.invoke({"query": last_message})
        state["analysis"] = analysis
        print("Query analysis:", analysis)
    except Exception as e:
        state["analysis"] = f"Analysis failed: {e}"
        print("Analysis failed:", e)
    return state


def retrieve_info_node(state: ResearchState) -> ResearchState:
    """Retrieve relevant chunks from session PDFs and long-term memory."""
    query = state["messages"][-1].content if state["messages"] else ""
    retrieved_texts: List[str] = []

    ### MODIFIED: Added session_id check and removed ephemeral vectorstore creation
    session_id = state.get("session_id")
    if not session_id:
        raise ValueError("session_id missing in state")

    session_pdf_store = SESSION_PDF_STORES.get(session_id) if session_id else None

    # Query short-term/session store
    if session_pdf_store:
        try:
            pdf_hits = session_pdf_store.similarity_search(query, k=3)
            retrieved_texts.extend([doc.page_content for doc in pdf_hits])
            print(f"Retrieved {len(pdf_hits)} docs from session PDFs.")
        except Exception as e:
            print("Session PDF retrieval error:", e)
    else:
        print("Warning: No session vectorstore found for session_id:", session_id)

    # Query long-term store
    try:
        lt_hits = long_term_vectorstore.similarity_search(query, k=3)
        retrieved_texts.extend([doc.page_content for doc in lt_hits])
        print(f"Retrieved {len(lt_hits)} docs from long-term memory.")
    except Exception as e:
        print("Long-term retrieval error (ok if empty):", e)

    return {**state, "retrieved_docs": retrieved_texts, "conversation_count": state.get("conversation_count", 0) + 1}


def generate_response_node(state: ResearchState) -> ResearchState:
    """Generate an answer: fuse history + retrieved docs + query."""
    query = state["messages"][-1].content
    context = "\n\n---\n\n".join(state.get("retrieved_docs", [])[:6])  # cap context length
    history = "\n".join([msg.content for msg in state["messages"][-6:]])  # keep last 6 messages

    system_prompt = (
        "You are a research assistant. Use the context and conversation history to answer the user's query.\n\n"
        "If the answer is not in the provided context, be honest and say you couldn't find it locally, "
        "but suggest reasonable next steps (e.g., which sections to check in the paper).\n\n"
        "Provide citations when possible in the form: (source: session_pdf_chunk_<n> or long_term_insight).\n\n"
        "Query: {query}\n\nContext: {context}\n\nHistory: {history}\n\nAnswer:"
    )
    prompt = ChatPromptTemplate.from_template(system_prompt)
    chain = prompt | llm | StrOutputParser()
    try:
        response = chain.invoke({"query": query, "context": context, "history": history})
    except Exception as e:
        response = "LLM generation failed: " + str(e)

    ai_msg = AIMessage(content=response)
    return {**state, "messages": state["messages"] + [ai_msg]}


def store_insights_node(state: ResearchState) -> ResearchState:
    """Extract concise insights from the last AI response and store to long-term memory."""
    if not state.get("messages"):
        return state

    last_response = state["messages"][-1].content
    prompt = ChatPromptTemplate.from_template(
        "Extract 1-3 concise, standalone insights from this assistant response. Output bullet lines separated by newline.\n\nResponse:\n{response}\n\nInsights:"
    )
    chain = prompt | llm | StrOutputParser()
    try:
        insights_text = chain.invoke({"response": last_response})
        insights = [s.strip() for s in insights_text.splitlines() if s.strip()][:3]
    except Exception as e:
        print("Insight extraction failed:", e)
        insights = []

    if insights:
        try:
            long_term_vectorstore.add_texts(insights)
            persist_long_term()
            print(f"Stored {len(insights)} insight(s) to long-term memory.")
        except Exception as e:
            print("Failed to store insights:", e)

    return {**state, "long_term_insights": insights}

In [30]:
# ========= Cell 6: Build the LangGraph workflow ==========
def create_research_graph():
    workflow = StateGraph(ResearchState)

    workflow.add_node("load_pdfs", load_pdfs_node)
    workflow.add_node("analyze_query", analyze_query_node)
    workflow.add_node("retrieve_info", retrieve_info_node)
    workflow.add_node("generate_response", generate_response_node)
    workflow.add_node("store_insights", store_insights_node)

    workflow.set_entry_point("load_pdfs")
    workflow.add_edge("load_pdfs", "analyze_query")
    workflow.add_edge("analyze_query", "retrieve_info")
    workflow.add_edge("retrieve_info", "generate_response")
    workflow.add_edge("generate_response", "store_insights")
    workflow.add_edge("store_insights", END)

    compiled = workflow.compile()
    return compiled

research_app = create_research_graph()
print("Research graph built successfully!")

Research graph built successfully!


In [31]:
# ========= Cell 7: Colab Upload / Run Example ==========
try:
    from google.colab import files
    is_colab = True
except Exception:
    is_colab = False

if is_colab:
    print("Running in Colab — you can upload PDFs now.")
    uploaded = files.upload()
    pdf_paths = list(uploaded.keys())
else:
    pdf_paths = []  # e.g., ["./paper1.pdf"]

# Create session_id once and include it in initial_state
session_id = str(uuid.uuid4())

initial_state: ResearchState = {
    "messages": [HumanMessage(content="What is the main topic of this paper?")],
    "pdf_paths": pdf_paths,
    "pdf_chunks": [],
    "retrieved_docs": [],
    "long_term_insights": [],
    "conversation_count": 0,
    "session_id": session_id
}

print("Starting session with id:", session_id)

# Run the agent
result = research_app.invoke(initial_state)

# Show results
if "messages" in result and result["messages"]:
    print("Agent Response:", result["messages"][-1].content)
else:
    print("Agent did not return a response. Check logs above for errors.")

print("Retrieved Docs (count):", len(result.get("retrieved_docs", [])))
print("Stored Insights:", result.get("long_term_insights", []))

Running in Colab — you can upload PDFs now.


Saving Resume.pdf to Resume.pdf
Starting session with id: c3d6dff8-a6bc-4f57-88b0-caa07f0284c1
Processed Resume.pdf: 4 chunks
Session PDF vectorstore created and cached (via add_texts).
load_pdfs_node: loaded 4 chunks across 1 PDF(s). session_id=c3d6dff8-a6bc-4f57-88b0-caa07f0284c1
Query analysis: keywords: main topic, paper; intent: to identify the primary subject or focus of the paper
Retrieved 3 docs from session PDFs.
Retrieved 0 docs from long-term memory.
Long-term memory persisted to: /content/long_term_memory
Stored 3 insight(s) to long-term memory.
Agent Response: Based on the provided context, the main topic of this paper appears to be the development and application of artificial intelligence (AI) and machine learning (ML) technologies, particularly focusing on chatbot systems and AI-driven customer support solutions. The paper discusses projects such as RAG Chatbot from user data, customer support AI agents, Instagram content automation, Telegram chatbots, and text-to-image

/tmp/ipython-input-531324258.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  long_term_vectorstore.persist()


In [32]:
# ========= Cell 8: Short Follow-up demo (short-term memory) ==========
if "messages" in result and result["messages"]:
    ### MODIFIED: Added session_id check and explicit preservation
    if not result.get("session_id"):
        raise ValueError("session_id missing in result state")
    follow_up_state = {
        **result,
        "messages": result["messages"] + [HumanMessage(content="Explain the key findings in a short paragraph.")],
        "session_id": result["session_id"]  # Explicitly ensure session_id is preserved
    }
    follow_up_result = research_app.invoke(follow_up_state)
    print("\nFollow-up Response:", follow_up_result["messages"][-1].content)
else:
    print("No messages in result; skipping follow-up.")

Processed Resume.pdf: 4 chunks
Using pre-existing session vectorstore from cache.
load_pdfs_node: loaded 4 chunks across 1 PDF(s). session_id=c3d6dff8-a6bc-4f57-88b0-caa07f0284c1
Query analysis: keywords: key findings, short paragraph; intent: request for a summary of main results in a brief paragraph
Retrieved 3 docs from session PDFs.
Retrieved 3 docs from long-term memory.
Long-term memory persisted to: /content/long_term_memory
Stored 3 insight(s) to long-term memory.

Follow-up Response: The key findings of the paper highlight significant advancements in AI and machine learning applications, particularly in developing chatbot systems and automation tools across various platforms. The projects demonstrate successful implementation of AI-driven solutions such as RAG chatbots, customer support agents, content automation for social media, and text-to-image generation, all contributing to improved efficiency and innovation in communication and service delivery. The results indicate str

In [33]:
# ========= Cell 9: Debug helper & session cleanup ==========
def debug_state(state: ResearchState):
    ### MODIFIED: Improved session_id reporting and cache check
    session_id = state.get("session_id", "Not set")
    print("Debug State:")
    print(f"  session_id: {session_id}")
    print(f"  Messages: {len(state.get('messages', []))}")
    print(f"  PDF Paths: {state.get('pdf_paths', [])}")
    print(f"  PDF Chunks: {len(state.get('pdf_chunks', []))} chunks")
    print(f"  Retrieved Docs: {len(state.get('retrieved_docs', []))}")
    print(f"  Long-Term Insights: {len(state.get('long_term_insights', []))}")
    print(f"  session_pdf_store_cached: {session_id in SESSION_PDF_STORES and SESSION_PDF_STORES[session_id] is not None}")
    print(f"  Conversation Count: {state.get('conversation_count', 0)}")

def cleanup_session(session_id: str):
    if session_id in SESSION_PDF_STORES:
        try:
            del SESSION_PDF_STORES[session_id]
            print(f"Session cache {session_id} removed.")
        except Exception as e:
            print("Error cleaning session cache:", e)

# Inspect the returned state
debug_state(result)

Debug State:
  session_id: c3d6dff8-a6bc-4f57-88b0-caa07f0284c1
  Messages: 2
  PDF Paths: ['Resume.pdf']
  PDF Chunks: 4 chunks
  Retrieved Docs: 3
  Long-Term Insights: 3
  session_pdf_store_cached: True
  Conversation Count: 1


In [34]:
# ========= Cell 10: Ask another query ==========
# Check if result has messages and session_id before proceeding
if "messages" in result and result["messages"] and result.get("session_id"):
    new_query_state = {
        **result,  # Carry forward the previous state (includes memory & session_id)
        "messages": result["messages"] + [HumanMessage(content="Summarize this paper in 3 bullet points.")],
        "session_id": result["session_id"]  # Explicitly preserve session_id
    }
    result2 = research_app.invoke(new_query_state)
    print("Agent Response:", result2["messages"][-1].content)
    print("Retrieved Docs (count):", len(result2.get("retrieved_docs", [])))
    print("Stored Insights:", result2.get("long_term_insights", []))
else:
    print("Error: Cannot proceed with Cell 8. Result state is missing messages or session_id.")

Processed Resume.pdf: 4 chunks
Using pre-existing session vectorstore from cache.
load_pdfs_node: loaded 4 chunks across 1 PDF(s). session_id=c3d6dff8-a6bc-4f57-88b0-caa07f0284c1
Query analysis: keywords: paper, summarize, bullet points; intent: request a concise summary of a paper in three bullet points
Retrieved 3 docs from session PDFs.
Retrieved 3 docs from long-term memory.
Long-term memory persisted to: /content/long_term_memory
Stored 3 insight(s) to long-term memory.
Agent Response: - The paper highlights various AI applications, including chatbot development (e.g., RAG Chatbot, Telegram bots), customer support AI agents, and content automation tools across platforms like Instagram, demonstrating advancements in automation and user engagement (source: session_pdf_chunk_3).

- It emphasizes the effectiveness of these AI solutions, with high performance scores (e.g., 5.0 out of 5.0), showcasing their success in solving complex problems and fostering innovation in digital communic